In [46]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the saved model and tokenizer
model_path = './saved_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Predefined order of labels
categories = [
    "Equity Financing", "Debt Financing", "Bankruptcy/financial distress",
    "New product launch", "Joint venture", "credit rating", 
    "financial results", "ESG announcement"
]

# Function to classify titles
def classify_titles(titles):
    results = []
    for title in titles:
        # Tokenize the title
        inputs = tokenizer(title, return_tensors="pt", truncation=True, padding=True)
        
        # Get model predictions
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get the predicted class
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()
        
        # Get the corresponding category
        predicted_category = categories[predicted_class_id]
        
        # Debugging: print logits and predicted class ID
        print(f"Title: '{title}'")
        print(f"Logits: {logits}")
        print(f"Predicted class ID: {predicted_class_id}")
        
        results.append((title, predicted_category))
    
    return results

# Titles to classify
titles_to_classify = ["Republican Senator Grassley launches new probe into Boeing, FAA"]

# Classify the titles
classified_titles = classify_titles(titles_to_classify)

# Print the results
for title, category in classified_titles:
    print(f"Title: '{title}' is classified as '{category}'")



Title: 'Republican Senator Grassley launches new probe into Boeing, FAA'
Logits: tensor([[ 0.1973, -0.1123, -0.4685,  0.4538,  0.0606, -0.5077,  0.0258,  0.0785]])
Predicted class ID: 3
Title: 'Republican Senator Grassley launches new probe into Boeing, FAA' is classified as 'New product launch'


In [40]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch

# Ensure GPU is visible
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Check if CUDA is available
print("CUDA is available" if torch.cuda.is_available() else "CUDA is not available")

# Load your dataset
data = pd.read_csv(r'C:\Users\admin\Desktop\SQ\try.csv')

# Predefined order of labels
categories = [
    "Equity Financing", "Debt Financing", "Bankruptcy/financial distress",
    "New product launch", "Joint venture", "credit rating", 
    "financial results", "ESG announcement"
]

# Map labels to integers based on predefined order
label_to_int = {label: i for i, label in enumerate(categories)}
int_to_label = {i: label for i, label in enumerate(categories)}

# Preprocess function
def preprocess_text(text):
    return text.lower().replace('\n', ' ').replace('\r', '')

# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)

# Encode labels using the predefined order
data['target'] = data['target'].map(label_to_int)

# Check class distribution
print(data['target'].value_counts())

# Split the dataset
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['target'], random_state=42)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_data.reset_index(drop=True))

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the texts
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Rename the target column to labels
train_dataset = train_dataset.rename_column("target", "labels")
test_dataset = test_dataset.rename_column("target", "labels")

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Load the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(categories))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./saved_model',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    weight_decay=0.01,
    no_cuda=not torch.cuda.is_available(),  # Use CUDA if available
    report_to=[]  
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Save the model
trainer.save_model('./saved_model')

# Optionally, save the tokenizer as well
tokenizer.save_pretrained('./saved_model')

# Function to classify new titles
def classify_titles(titles):
    results = []
    for title in titles:
        # Tokenize the title
        inputs = tokenizer(title, return_tensors="pt", truncation=True, padding=True)
        
        # Get model predictions
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get the predicted class
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()
        
        # Get the corresponding category
        predicted_category = int_to_label[predicted_class_id]
        
        # Debugging: print logits and predicted class ID
        print(f"Title: '{title}'")
        print(f"Logits: {logits}")
        print(f"Predicted class ID: {predicted_class_id}")
        
        results.append((title, predicted_category))
    
    return results

# Classify the titles
titles_to_classify = ["Company XYZ Signs $40 Million Loan Agreement", "DEF Corp Obtains New Line of Credit", "XYZ Ltd Faces Bankruptcy Due to Losses"]
classified_titles = classify_titles(titles_to_classify)

# Print the results
for title, category in classified_titles:
    print(f"Title: '{title}' is classified as '{category}'")


CUDA is not available
target
0    7
1    7
2    4
3    4
4    4
5    4
6    4
7    4
Name: count, dtype: int64


Map: 100%|██████████| 8/8 [00:00<00:00, 1979.61 examples/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
  3%|▎         | 4/120 [00:01<00:35,  3.22it/s]

{'eval_loss': 2.062715768814087, 'eval_runtime': 0.0404, 'eval_samples_per_second': 197.971, 'eval_steps_per_second': 24.746, 'epoch': 1.0}


  7%|▋         | 8/120 [00:02<00:36,  3.09it/s]

{'eval_loss': 2.0294554233551025, 'eval_runtime': 0.0339, 'eval_samples_per_second': 235.925, 'eval_steps_per_second': 29.491, 'epoch': 2.0}


...

{'eval_loss': 0.8950214385986328, 'eval_runtime': 0.0449, 'eval_samples_per_second': 178.25, 'eval_steps_per_second': 22.281, 'epoch': 29.0}


100%|██████████| 120/120 [00:49<00:00,  2.42it/s]


{'eval_loss': 0.8934305906295776, 'eval_runtime': 0.0439, 'eval_samples_per_second': 182.294, 'eval_steps_per_second': 22.787, 'epoch': 30.0}
{'train_runtime': 49.6654, 'train_samples_per_second': 18.121, 'train_steps_per_second': 2.416, 'train_loss': 1.1097442626953125, 'epoch': 30.0}


100%|██████████| 1/1 [00:00<00:00, 1001.98it/s]


Evaluation results: {'eval_loss': 0.8934305906295776, 'eval_runtime': 0.0464, 'eval_samples_per_second': 172.45, 'eval_steps_per_second': 21.556, 'epoch': 30.0}
Title: 'Company XYZ Signs $40 Million Loan Agreement'
Logits: tensor([[ 0.0801,  2.5225, -0.3319, -0.3793, -0.8046, -0.6057, -0.4809, -0.9924]])
Predicted class ID: 1
Title: 'DEF Corp Obtains New Line of Credit'
Logits: tensor([[-0.0238,  2.4712, -0.3280, -0.3403, -0.8235, -0.5594, -0.4576, -1.0544]])
Predicted class ID: 1
Title: 'XYZ Ltd Faces Bankruptcy Due to Losses'
Logits: tensor([[-0.0292, -0.0510,  1.8050, -0.7112, -0.4422, -0.4796, -0.5167, -0.6468]])
Predicted class ID: 2
Title: 'Company XYZ Signs $40 Million Loan Agreement' is classified as 'Debt Financing'
Title: 'DEF Corp Obtains New Line of Credit' is classified as 'Debt Financing'
Title: 'XYZ Ltd Faces Bankruptcy Due to Losses' is classified as 'Bankruptcy/financial distress'


In [34]:
import os
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# Ensure GPU is visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Check if CUDA is available
print("CUDA is available" if torch.cuda.is_available() else "CUDA is not available")

# Load your dataset
data = pd.read_csv(r'C:\Users\admin\Desktop\SQ\try.csv')

# Preprocess function
def preprocess_text(text):
    return text.lower().replace('\n', ' ').replace('\r', '')

# Apply preprocessing
data['text'] = data['text'].apply(preprocess_text)

# Encode labels
label_encoder = LabelEncoder()
data['target'] = label_encoder.fit_transform(data['target'])
num_labels = len(label_encoder.classes_)

# Check class distribution
print(data['target'].value_counts())

# Split the dataset
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['target'], random_state=42)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_data.reset_index(drop=True))

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/roberta-large-zeroshot-v2.0-c")

# Tokenize the texts
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Rename the target column to labels
train_dataset = train_dataset.rename_column("target", "labels")
test_dataset = test_dataset.rename_column("target", "labels")

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/roberta-large-zeroshot-v2.0-c", num_labels=num_labels, ignore_mismatched_sizes=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./saved_model',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    no_cuda=not torch.cuda.is_available(),  # Use CUDA if available
    report_to=[]  
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Save the model
trainer.save_model('./saved_model')

# Optionally, save the tokenizer as well
tokenizer.save_pretrained('./saved_model')

# Function to classify new titles
def classify_titles(titles):
    results = []
    for title in titles:
        # Tokenize the title
        inputs = tokenizer(title, return_tensors="pt", truncation=True, padding=True)
        
        # Get model predictions
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get the predicted class
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()
        
        # Get the corresponding category
        predicted_category = label_encoder.inverse_transform([predicted_class_id])[0]
        
        # Debugging: print logits and predicted class ID
        print(f"Title: '{title}'")
        print(f"Logits: {logits}")
        print(f"Predicted class ID: {predicted_class_id}")
        
        results.append((title, predicted_category))
    
    return results

# Classify the titles
titles_to_classify = ["Company XYZ Signs $40 Million Loan Agreement", "DEF Corp Obtains New Line of Credit", "XYZ Ltd Faces Bankruptcy Due to Losses"]
classified_titles = classify_titles(titles_to_classify)

# Print the results
for title, category in classified_titles:
    print(f"Title: '{title}' is classified as '{category}'")


CUDA is not available
target
3    7
1    7
0    4
5    4
4    4
6    4
7    4
2    4
Name: count, dtype: int64


Map: 100%|██████████| 8/8 [00:00<00:00, 2650.43 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at MoritzLaurer/roberta-large-zeroshot-v2.0-c and are newly initialized because the shapes did not match:
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.out_proj.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([8, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\admin\Desktop\traini\autotrain-env\Lib\site-packages\transformers\training_args.py:1489: FutureWarning: using `no_c

{'eval_loss': 2.16719126701355, 'eval_runtime': 0.1891, 'eval_samples_per_second': 42.296, 'eval_steps_per_second': 5.287, 'epoch': 1.0}


                                              
 20%|██        | 8/40 [00:15<01:01,  1.94s/it]

{'eval_loss': 2.142174482345581, 'eval_runtime': 0.2555, 'eval_samples_per_second': 31.317, 'eval_steps_per_second': 3.915, 'epoch': 2.0}


...

{'eval_loss': 1.7573444843292236, 'eval_runtime': 0.269, 'eval_samples_per_second': 29.737, 'eval_steps_per_second': 3.717, 'epoch': 9.0}


                                               
100%|██████████| 40/40 [01:26<00:00,  2.16s/it]


{'eval_loss': 1.7503397464752197, 'eval_runtime': 0.258, 'eval_samples_per_second': 31.013, 'eval_steps_per_second': 3.877, 'epoch': 10.0}
{'train_runtime': 86.3574, 'train_samples_per_second': 3.474, 'train_steps_per_second': 0.463, 'train_loss': 1.8576419830322266, 'epoch': 10.0}


100%|██████████| 1/1 [00:00<00:00, 1002.94it/s]


Evaluation results: {'eval_loss': 1.7503397464752197, 'eval_runtime': 0.2576, 'eval_samples_per_second': 31.062, 'eval_steps_per_second': 3.883, 'epoch': 10.0}
Title: 'Company XYZ Signs $40 Million Loan Agreement'
Logits: tensor([[-0.7195,  1.2879, -0.4163,  0.2946, -0.7942, -0.8786, -0.6896, -1.1208]])
Predicted class ID: 1
Title: 'DEF Corp Obtains New Line of Credit'
Logits: tensor([[-0.7813,  1.1869, -0.3781,  0.2787, -0.7756, -0.8638, -0.6422, -1.1289]])
Predicted class ID: 1
Title: 'XYZ Ltd Faces Bankruptcy Due to Losses'
Logits: tensor([[ 0.8869, -0.5310, -0.7760,  0.2495,  0.1456,  0.1347, -0.4021,  0.1975]])
Predicted class ID: 0
Title: 'Company XYZ Signs $40 Million Loan Agreement' is classified as 'Debt Financing'
Title: 'DEF Corp Obtains New Line of Credit' is classified as 'Debt Financing'
Title: 'XYZ Ltd Faces Bankruptcy Due to Losses' is classified as 'Bankruptcy/financial distress'
